<a href="https://colab.research.google.com/github/AI4GoodE1/AI4GoodE1/blob/main/distilBERTmodel/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
#imports
import numpy as np
import pandas as pd
# import torch
import tensorflow as tf
from sklearn.model_selection import train_test_split

#transformers import
from transformers import DistilBertTokenizerFast, DistilBertModel
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments


In [ ]:
# Get data
df = pd.read_csv('../dataset/merged_dataset_preprocessed.csv')


In [ ]:
df2 = df[df.isna().any(axis=1)]
df2.index

In [ ]:
df2 = df.drop(index = df2.index)
df3 = df2[df2.isna().any(axis=1)]

In [ ]:
df3

In [ ]:
X = list(df2["Text"])
y =list(df2["Class"])

X_Train, X_Test, y_Train, y_Test = train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(X_Train, truncation=True, padding=True)
# print(train_encodings)

In [ ]:
test_encodings = tokenizer(X_Test, truncation=True, padding=True)
test_encodings

In [40]:

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_Train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_Test
))

KeyboardInterrupt: 

In [ ]:
# setting most of these features to one further reduces runtime
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=4,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    eval_steps = 10
)

In [ ]:
# next(iter(train_dataloader))

In [ ]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
trainer.predict(test_dataset)

In [ ]:
trainer.save_model('Transformer_Bert_Model')

In [ ]:
trainer.predict(test_dataset)[1].shape

In [ ]:
output=trainer.predict(test_dataset)[1]

In [ ]:
from sklearn.metrics import confusion_matrix
confusionMatrixResult=confusion_matrix(y_Test,output)
tn, fp, fn, tp = confusionMatrixResult.ravel()
print("Confusion Matrix results:")
print(confusionMatrixResult)
print(f"Number of Correctly Predicted Fraud Emails: {tp}")
print(f"Number of Falsely Predicted Fraud Emails: {fp}")
print(f"Number of Correctly Predicted Legit Emails: {tn}")
print(f"Number of Falsely Predicted Fraud Emails: {fn}")

In [ ]:
test_email1 = "Hi team, let’s use my webex for our meeting at 3:15 again. Here is the link: https://rbcteams.webex.com/meet/test.test Thanks"
test_email1 = tokenizer(test_email1, truncation=True, padding=True, return_tensors="tf")
#print(test_email1)
#test_email1 = tf.data.Dataset.from_tensor_slices(test_email1)
#print(test_email1)
output = model(test_email1)
print(output.loss)
print(f"trained: {output.logits}")
model_untrained = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
output_untrained = model_untrained(test_email1)
print(output_untrained.loss)
print(f"untrained: {output_untrained.logits}")